# Regression Model for the Prediction of Patient Pay

This model is based on the HistGradientBoostingRegressor in sklearn
HistGradientBoostingRegressor has native support for categorical variables allowing for minimal preprocessing of the features in the data set. The categorical variables for insurance (bin, pcn, and group) and drug are transformed into ordinal values using the label encoder.

Cross validation of model fitted only on the insurance information and drug name show a mean average precentage error of ~1.5%.


### Relevant libraries are imported.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import KFold, cross_val_score, cross_validate

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

### Data set is imported and split into training and test sets.

Before the data is split into training and test sets, duplicate instances are removed from the data set to prevent data leakage. Rejected claims are also removed from the data set as rejected claims do not offer any information on patient pay. 

In [5]:
cmm_data = pd.read_csv('/home/ruggiec/Downloads/pharmacy_tx.csv')

cmm_data_no_dups = cmm_data.drop_duplicates()

cmm_data = cmm_data[cmm_data.rejected == 0].copy()

cmm_train, cmm_test = train_test_split(cmm_data, shuffle=True, 
                             random_state= 614, test_size=0.2)

### Rejected claims are removed from the data set

Rejected claims do not offer any information on patient pay.

In [3]:
cmm_paid = cmm_train.copy()

### Minimal feature engineering is perfomed

The month information is extracted from the 'tx_date' column and all categorical variables are encoded using the OrdinalEncoder.

In [4]:
cmm_paid['tx_date'] = pd.to_datetime(cmm_paid.tx_date)
cmm_paid['month'] = cmm_paid.tx_date.dt.month
cmm_paid['day_of_year'] = cmm_paid.tx_date.dt.dayofyear

In [5]:
enc = OrdinalEncoder()

In [6]:
encoded = enc.fit_transform(cmm_paid[['diagnosis', 'drug', 'bin', 'pcn', 'group']])
cmm_paid['diagnosis_encoded'] = encoded[:,0]
cmm_paid['drug_encoded'] = encoded[:,1]
cmm_paid['bin_encoded'] = encoded[:,2]
cmm_paid['pcn_encoded'] = encoded[:,3]
cmm_paid['group_encoded'] = encoded[:,4]

### Model Perfomance

Five models are trained below. The second model shows the importance of defining categorical features in the HistGradientBoostingRegressor.

In [7]:
## Model 1
## Baseline model using only insurance info and the drug name.

features = ['bin_encoded', 'pcn_encoded', 'group_encoded', 'drug_encoded'] # features
hgbr = HistGradientBoostingRegressor(random_state=412)
base_scores = cross_validate(hgbr, cmm_paid[features], cmm_paid.patient_pay, 
                         cv=5, scoring=['r2', 'neg_root_mean_squared_error', 
                                        'neg_mean_absolute_percentage_error'])

In [8]:
## Model 2
## Improved model using only insurance info with defined categorical features.

features = ['bin_encoded', 'pcn_encoded', 'group_encoded', 'drug_encoded'] # features
hgbr = HistGradientBoostingRegressor(random_state=412, categorical_features=[0, 1, 2, 3])
cat_scores = cross_validate(hgbr, cmm_paid[features], cmm_paid.patient_pay, 
                         cv=5, scoring=['r2', 'neg_root_mean_squared_error', 
                                        'neg_mean_absolute_percentage_error'])

In [9]:
## Model 3
## This model is the same as the model 2 with no bound on the number of leaf nodes.
## Training this model is slower (order of minutes) but shows greatly increased metrics.)

features = ['bin_encoded', 'pcn_encoded', 'group_encoded', 'drug_encoded'] # features
hgbr = HistGradientBoostingRegressor(random_state=412, categorical_features=[0, 1, 2, 3], max_leaf_nodes=None)
scores = cross_validate(hgbr, cmm_paid[features], cmm_paid.patient_pay, 
                         cv=5, scoring=['r2', 'neg_root_mean_squared_error', 
                                        'neg_mean_absolute_percentage_error'])

In [10]:
## Model 4
## This model includes day of year as a feature with no bound on the number of leaf nodes.
## Training this model is slower (order of minutes) but shows greatly increased metrics.)

features = ['bin_encoded', 'pcn_encoded', 'group_encoded', 'drug_encoded', 'day_of_year'] # features
hgbr = HistGradientBoostingRegressor(random_state=412, categorical_features=[0, 1, 2, 3], max_leaf_nodes=None)
scores_day = cross_validate(hgbr, cmm_paid[features], cmm_paid.patient_pay, 
                         cv=5, scoring=['r2', 'neg_root_mean_squared_error', 
                                        'neg_mean_absolute_percentage_error'])

In [11]:
## Model 5
## This model includes month as a feature with no bound on the number of leaf nodes.
## Training this model is slower (order of minutes) but shows greatly increased metrics.)

features = ['bin_encoded', 'pcn_encoded', 'group_encoded', 'drug_encoded', 'month'] # features
hgbr = HistGradientBoostingRegressor(random_state=412, categorical_features=[0, 1, 2, 3, 4], max_leaf_nodes=None)
scores_month = cross_validate(hgbr, cmm_paid[features], cmm_paid.patient_pay, 
                         cv=5, scoring=['r2', 'neg_root_mean_squared_error', 
                                        'neg_mean_absolute_percentage_error'])

### Comparrison of Models

In [12]:
column_names = ['Model', 'Mean Training Time (s)', 'Mean Scoring Time (s)', 'Mean R2', 
                'Mean RMSE', 'Mean MAPE']
model_metrics = [base_scores, cat_scores, scores, scores_day, scores_month]
summary = np.zeros((5, 6))
i,j = 0,1
for metric in model_metrics:
    for value in metric.values():
        summary[i, j] = value.mean()
        j+=1
    j=1
    i+=1
    
summary_metrics = pd.DataFrame(summary, columns=column_names)
summary_metrics['Model'] = ['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Model 5']
summary_metrics['Mean RMSE'] = summary_metrics['Mean RMSE'] * -1
summary_metrics['Mean MAPE'] = summary_metrics['Mean MAPE'] * -100
summary_metrics

,Model,Mean Training Time (s),Mean Scoring Time (s),Mean R2,Mean RMSE,Mean MAPE
0,Model 1,15.865910,1.872885,0.941909,9.761691,32.537215
1,Model 2,18.169800,3.077982,0.992623,3.478517,9.392643
2,Model 3,108.165624,9.659096,0.996171,2.506311,1.588043
3,Model 4,490.725664,8.509540,0.997207,2.140481,1.507308
4,Model 5,286.502942,9.962141,0.997168,2.155320,1.418522


Model 4 and Model 5 show very similar statistics. Model 5 outperforms Model 4 in two key areas training time and mean MAPE.

### Build Pipeline

Pipeline allows for the preprocessing and fitting/predicting on the data with a single object. This may be useful for building an application with the model or just predicting on the test set.

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

class MonthTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(Self, X, y = None):
        X = pd.to_datetime(X)
        X = X.dt.month.values.reshape(-1,1)
        return X

In [15]:
ct1 = ColumnTransformer(transformers=[('mt', MonthTransformer(), 0)], remainder='passthrough')
ct2 = ColumnTransformer(transformers=[('enc', OrdinalEncoder(), 
                                       [1, 2, 3, 4])], remainder='passthrough')

In [16]:
hgbr_pipe = Pipeline([('months_column', ct1), ('remainder_columns', ct2), 
                      ('hgbr', HistGradientBoostingRegressor(categorical_features=[0,1,2,3,4], 
                                                             max_leaf_nodes=None))])

In [17]:
X = cmm_paid[['tx_date', 'bin', 'pcn', 'group', 'drug']].copy()

In [18]:
y = cmm_paid['patient_pay'].copy()

In [19]:
hgbr_pipe.fit(X, y)

Pipeline(steps=[('months_column',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('mt', MonthTransformer(),
                                                  0)])),
                ('remainder_columns',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enc', OrdinalEncoder(),
                                                  [1, 2, 3, 4])])),
                ('hgbr',
                 HistGradientBoostingRegressor(categorical_features=[0, 1, 2, 3,
                                                                     4],
                                               max_leaf_nodes=None))])

In [20]:
predicted = hgbr_pipe.predict(X)

In [21]:
print('MAPE:', mean_absolute_percentage_error(y, predicted)*100)
print('MSE:', np.sqrt(mean_squared_error(y, predicted)))

MAPE: 1.3915318667123635
MSE: 2.128186582666257


### Check Model on the Test Data

In [22]:
cmm_test_paid = cmm_test[cmm_test.rejected == 0].copy()
X_test = cmm_test_paid[['tx_date', 'bin', 'pcn', 'group', 'drug']].copy()
y_test = cmm_test_paid['patient_pay'].copy()

In [24]:
predicted_test = hgbr_pipe.predict(X_test)
print('MAPE:', mean_absolute_percentage_error(y_test, predicted_test)*100)
print('MSE:', np.sqrt(mean_squared_error(y_test, predicted_test)))

MAPE: 1.405221196056351
MSE: 2.148680579833276


The HGBR model does quite well with a minimal number of inputs.

### Save Model and Model Parameters

In [105]:
import pickle as pkl

In [42]:
#with open('hgbr_model', 'wb') as f:
#    pkl.dump(hgbr_pipe, f)

#with open('hgbr_model', 'rb') as f:
#    cls = pkl.load(f)

In [15]:
#with open('hgbr_params', 'wb') as f:
#    pkl.dump(hgbr_pipe.get_params(deep=True), f)
    
#with open('hgbr_params', 'rb') as f:
#    parameters = pkl.load(f)

### Prediction Interval

https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-quantile-py

In [25]:
## cleaned up data to play with.

X

,tx_date,bin,pcn,group,drug
10120050,2022-09-29,664344,T17LNK,Z01MLD4I,branded hozirol
687578,2022-01-21,725700,NaN,DYGBI610ZY,branded prazinib
9300050,2022-09-09,322463,T52GV,EVD4X5,branded plazamiglutic
631063,2022-01-20,571569,KB38N,6BYJBW,branded antimab
8642802,2022-08-23,664344,BIZF,QK6BI1N61,branded tanoclolol
...,...,...,...,...,...
12669254,2022-11-29,96934,S76J7V6,NaN,branded prazinib
2653304,2022-03-19,96934,S76J7V6,NaN,generic pucomalol
3478433,2022-04-12,322463,NaN,HO8HUGL,generic glycontazepelol
9362180,2022-09-11,664344,NaN,STGRDKR1J5RD,branded tafistitrisin


In [26]:
y

10120050    17.44
687578      11.22
9300050      8.21
631063      23.26
8642802     11.62
            ...  
12669254    13.93
2653304     13.93
3478433      4.21
9362180     10.77
7466679      4.00
Name: patient_pay, Length: 10258707, dtype: float64

In [27]:
X_pinterval = ct2.fit_transform(ct1.fit_transform(X))

In [28]:
X_pinterval

array([[5.0, 35.0, 45.0, 31.0, 9],
       [8.0, nan, 15.0, 52.0, 1],
       [2.0, 36.0, 16.0, 50.0, 9],
       ...,
       [2.0, nan, 20.0, 89.0, 4],
       [5.0, nan, 36.0, 66.0, 9],
       [0.0, 9.0, 41.0, 89.0, 7]], dtype=object)

In [29]:
all_models = {}
for alpha in [0.05, 0.5, 0.95]:
    hgbr = HistGradientBoostingRegressor(loss='quantile', quantile=alpha, 
                                         categorical_features=[0,1,2,3,4], max_leaf_nodes=None, early_stopping=False)
    all_models["q %1.2f" % alpha] = hgbr.fit(X_pinterval, y)

In [30]:
hgbr = HistGradientBoostingRegressor(categorical_features=[0,1,2,3,4], max_leaf_nodes=None, early_stopping=False)
all_models['mse'] = hgbr.fit(X_pinterval, y)

The code below trains 4 modesl and evaluates them using the pinball loss and mse metric.

In [31]:
from sklearn.metrics import mean_pinball_loss

def highlight_min(x):
    x_min = x.min()
    return ["font-weight: bold" if v == x_min else "" for v in x]


results = []
for name, hgbr in sorted(all_models.items()):
    metrics = {"model": name}
    y_pred = hgbr.predict(X_pinterval)
    for alpha in [0.05, 0.5, 0.95]:
        metrics["pbl=%1.2f" % alpha] = mean_pinball_loss(y, y_pred, alpha=alpha)
    metrics["MSE"] = mean_squared_error(y, y_pred)
    results.append(metrics)

pd.DataFrame(results).set_index("model").style.apply(highlight_min)

,pbl=0.05,pbl=0.50,pbl=0.95,MSE
model,,,,
mse,0.244352,0.244352,0.244352,4.522234
q 0.05,0.546525,5.457585,10.368644,1277.965075
q 0.50,0.175966,0.319318,0.462670,32.382004
q 0.95,4.047479,2.144000,0.240520,47.724327


In [53]:
## true price

y.iloc[35220]

4.21

In [65]:
## price prediction interval 

print('The price typically ranges between', all_models['q 0.05'].predict([X_pinterval[35220,:]])[0], 'and',
      all_models['q 0.95'].predict([X_pinterval[35220,:]])[0])

The price typically ranges between 4.20262890784693 and 15.427415157841635


In [55]:
## quantile 0.5 prediction 

all_models['q 0.50'].predict([X_pinterval[35220,:]])[0]

4.204134193790747

In [56]:
## mse prediction

all_models['mse'].predict([X_pinterval[35220,:]])[0]

4.210581421222192

For the this example both the median and mse models predict very well while the prediction invterval is skewed to the high end of pay.